# Markdown Pricing Optimization

<img src='https://drive.google.com/uc?id=1GuUfN0BNWUoDRUIWAae8DAugc-rqdGTj' width="600" height="400">  
We will build a basic optimization model to show the application of the Pyomo package.  
Then, complicate the model to simultaneously solve the markdown pricing of more products.  
By doing so, we can determine the ideal pricing strategy based on the given information.

#Set-Up

In [ ]:
# Import libraries
!pip install -q pyomo
from pyomo.environ import *

!apt-get install -y -qq glpk-utils


# Using the solver
from pyomo.opt import SolverFactory


import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 128048 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

#Code that solves one instance of the Markdown Pricing Problem

In [ ]:
# Inputs
list_price = 55.48 # Undiscounted price
demand = [120, 160, 194, 223] # For 0%, 15%, 30%, 50% discounts, respectively
weeksleft = 15 # Current # of weeks left in the selling season
inv = 1421 # Current amount of inventory left
sal = 13.87 # Salvage value per unit

In [ ]:
# Calculate discounted prices
price = [list_price, list_price * (1-.15), list_price * (1-.3), list_price * (1-.5)] #For 0%, 15%, 30%, 50% discounts, respectively

# Definie the optimization model
model = ConcreteModel()
model.x = Var(range(4), domain=NonNegativeReals)
model.Objective = Objective(expr = sum(demand[i] * model.x[i] * (price[i] - sal) for i in range(4)) + sal * inv, sense = maximize)
model.InvConstraint = Constraint(expr = sum(demand[i] * model.x[i] for i in range(4)) <= inv)
model.WeeksConstraint = Constraint(expr = sum(model.x[i] for i in range(4)) <= weeksleft)
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

1 Var Declarations
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 4993.2*x[0] + 5326.08*x[1] + 4843.404*x[2] + 3093.0099999999998*x[3] + 19709.27

2 Constraint Declarations
    InvConstraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                                      : Upper  : Active
        None :  -Inf : 120*x[0] + 160*

In [ ]:
# Solve the model
opt = SolverFactory('glpk')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 78837.08, 'Upper bound': 78837.08, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 9, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.010934829711914062}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
# Print results
print('Optimal scheduled time at each price level over the remaining weeks: ', (model.x[0](), model.x[1](), model.x[2](), model.x[3]()))
print('Projected optimal total revenue: ', model.Objective())

Optimal scheduled time at each price level over the remaining weeks:  (11.8416666666667, 0.0, 0.0, 0.0)
Projected optimal total revenue:  78837.08000000016


# Solve multiple projects

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving MP_scenarios.xlsx to MP_scenarios.xlsx


In [ ]:
df = pd.read_excel(open('MP_scenarios.xlsx','rb'), sheet_name = 'Sheet1') 
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245


In [ ]:
# Adjust data to list format
price = df.Price
demand0 = df['Demand (0% discount)']
demand15 = df['Demand (15% discount)']
demand30 = df['Demand (30% discount)']
demand50 = df['Demand (50% discount)']
weeksleft = df['Weeks Left']
inv = df['Inventory Remaining']
sal = df['Salvage Value']

# Set empty lists to store our results
obj = []
solution = []

In [ ]:
for i in range(len(price)):
    # DVs
    n = 4

    # Define our concrete model
    model = ConcreteModel()

    # Declare the decision variables
    model.x = Var(range(n), domain = NonNegativeReals)

    # Constraints
    model.InvConstraint = Constraint(expr = (demand0[i] * model.x[0]
                                              +demand15[i] * model.x[1]
                                              +demand30[i] * model.x[2]
                                              +demand50[i] * model.x[3]) <= inv[i])
    model.WeeksConstraint = Constraint(expr = sum(model.x[j] for j in range(n)) <= weeksleft[i])

    # Set the objective
    model.Objective = Objective(expr = price[i] * demand0[i] * model.x[0]
                                          + 0.85*price[i] * demand15[i] * model.x[1]
                                          + 0.70*price[i] * demand30[i] * model.x[2]
                                          + 0.50*price[i] * demand50[i] * model.x[3]
                                          + (sal[i] * (inv[i] - (demand0[i] * model.x[0] + demand15[i] * model.x[1] + demand30[i] * model.x[2] + demand50[i] * model.x[3]))), sense = maximize)

    # Specify the solver, and solve
    opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
    result = opt.solve(model)

    # Store the optimal objective
    obj.append(model.Objective())

    # Store the optimal decision variables
    for k in range(n):
        solution.append(value(model.x[k]))


In [ ]:
ss = list(np.array_split(solution, len(df)))

In [ ]:
# Add 2 columns to our dataframe and print out our solutions.
df['pricing plan'] = ss
df['revenue'] = obj
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,pricing plan,revenue
0,1,55.48,120,160,194,223,15,1421,13.870,"[11.8416666666667, 0.0, 0.0, 0.0]",78837.080000
1,2,53.68,115,149,171,197,15,2396,13.420,"[0.0, 15.0, 0.0, 0.0]",104139.200000
2,3,61.56,140,191,207,459,15,2544,15.390,"[6.29411764705882, 8.70588235294118, 0.0, 0.0]",141254.127529
3,4,65.72,115,151,242,278,15,1316,16.430,"[11.4434782608696, 0.0, 0.0, 0.0]",86487.520000
4,5,64.98,120,173,221,335,15,1377,16.245,"[11.475, 0.0, 0.0, 0.0]",89477.460000
